In [1]:
# import packages
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import os
import urllib
import pyodbc as odbc
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [2]:
# BBLs of 530 transferred props from TPT Rounds 1-9
transfers = [2025540008, 2025540002, 2025660001, 2027650290, 2026080001, 2026120001, 2026090035, 2026920074, 2025500007, 2026120035, 2025500017, 2026760085, 2026850050, 2026850051, 2027000053, 2026900126, 2026000197, 2027610158, 2026510017, 2027630107, 2025620036, 2026750045, 2025620070, 2025620069, 2026610005, 2026620029, 2026120006, 2026200005, 2026950052, 2027450030, 2027520046, 2026200052, 2026200050, 2026130023, 2026690025, 2026200049, 2026200056, 2027170067, 2027620061, 2026120003, 2026910075, 2026220060, 2026790051, 2026900131, 2026900147, 2026900146, 2026900145, 2027500003, 3018320025, 3018330080, 3018240043, 3018300069, 3018320091, 3013390077, 3013390002, 3012400024, 3012400008, 3012400004, 3018320051, 3018420017, 3018600092, 3018320046, 3017560026, 3014830059, 3034130066, 3034200042, 3015030012, 3015030013, 3013710051, 3035570004, 3051900050, 3008400010, 3011550057, 3020170063, 3018510004, 3018380055, 3011550070, 3018600059, 3034870065, 3022380007, 3027000004, 3019900003, 2029350025, 2026630059, 2029270054, 2031190058, 2023260028, 2024040022, 2029680028, 2029680035, 2026220030, 1017180062, 1019270021, 1019270049, 1021200065, 1017180015, 1019430027, 1017210024, 1017170046, 1019270139, 1018280122, 1021070010, 1018360020, 1018360019, 1018210008, 1018270022, 1018310043, 1019660046, 1020360022, 1017290047, 1017350033, 1020060025, 1020060024, 1020360064, 1019130009, 1019130010, 1019520043, 1019660047, 1017290019, 1017830029, 1019520051, 1015580033, 1017210014, 1017210017, 1021070020, 1018210014, 1020360010, 1021200028, 1017170069, 1020880074, 1020880114, 1018360041, 1018360043, 1018460059, 1018450007, 1018230010, 1018300029, 1018450014, 1020540022, 1017720150, 1017720050, 3003090001, 3009590007, 3004410005, 3001980001, 3005970016, 3033060023, 3050720012, 3051890004, 3032690052, 3033260048, 3032200040, 3033260026, 3033260029, 3032890021, 3032200051, 3032460050, 3031730051, 3031750038, 3031750037, 3019790010, 3019820014, 3046280001, 3046120010, 3045970052, 3046050019, 3051360059, 3052300072, 3048960003, 3012540007, 3050900059, 3050900051, 3012070004, 3048700041, 3016380076, 3016380074, 3031660051, 3031570052, 3031650023, 3007810075, 3014330043, 3016220042, 3021140032, 3014330028, 3045980079, 3014640025, 3014530019, 3014400008, 3015280023, 3018470037, 3015170002, 3015170007, 3015190053, 3015330030, 3018260022, 3016320002, 3016270048, 3015080065, 3015080053, 3014850015, 3014850017, 3014850020, 3034300027, 3034360019, 3033990015, 3032950016, 3034300032, 3032970040, 3031890020, 3015080006, 3016830079, 3012080047, 3012540006, 3006850053, 3007810074, 3007650056, 3007650006, 3008850046, 3008850048, 3005180007, 3007190045, 3016830083, 3024070026, 3030400024, 3030080006, 3050260325, 3040070013, 3040200024, 3017890080, 3017580049, 3017950015, 3042110020, 3037850001, 3037850075, 3018060041, 3012750043, 3017390004, 3015880038, 3017690041, 3015990004, 3017760044, 3018030076, 3012150070, 3018320066, 3016300040, 3016510018, 3012100039, 3018650038, 3013770054, 3020150066, 3012170068, 3012050004, 3011680082, 3011760090, 3011620016, 3011870058, 3011870059, 3019250027, 3013900077, 3013860070, 3044560054, 3039980011, 3042110019, 3042340027, 3043100002, 3044540040, 3051280001, 3052120043, 3050470006, 3019970020, 3034310022, 3011680065, 3015090052, 3033130004, 3012590054, 3011880055, 3017670035, 3012330016, 3018010008, 3040880035, 3040890012, 3014510069, 3015130025, 3039420005, 3037760050, 3014510046, 3014510051, 3036130047, 3012330047, 3086760046, 3070620008, 3009590002, 3000290258, 3020460009, 3013720055, 3012020059, 3017860060, 3016250067, 3086620238, 3042340023, 3016470019, 3017080037, 3036280025, 3050440063, 3033300019, 3034010037, 3032090012, 3032890044, 3032460038, 3031750013, 3034560019, 3033890045, 3032870040, 3032630051, 3014760001, 3016530058, 3016720004, 3031660004, 3040890133, 3044570063, 3016380078, 3013680001, 3013810036, 3015040014, 3018170036, 3014660065, 3014660067, 3014700029, 3014520051, 3032080050, 3014330039, 3053420030, 3083070069, 3067630046, 3053420028, 3012230001, 3017670068, 3012230051, 3012230052, 3012350071, 3014830058, 3016560048, 3018230029, 3018230031, 3010430021, 3017880053, 3018250041, 3019480014, 3019870071, 2024300035, 2024330043, 2024340040, 2024380080, 2024270040, 2026330002, 2026330003, 2048480039, 2048730037, 2048730038, 2046610027, 2030310053, 2031970009, 2032120092, 2046610054, 2024320075, 2025160026, 2028690087, 2025080026, 2025640005, 2039630035, 2041740036, 2032900058, 2025280027, 2028720028, 2028760028, 2025090056, 2025220121, 2029700013, 2024610067, 2024330067, 2024330073, 2024560030, 2027910052, 2029790006, 2027910054, 2031420070, 2030370065, 2030560017, 2031050004, 2048360078, 2032250022, 2028780061, 2048550057, 2046260024, 2048480030, 2029580066, 2030810006, 2026620028, 2034540020, 2031110035, 2029350001, 2023730043, 2026950040, 2027230029, 2025240020, 2025210001, 2025340038, 2028610114, 2028610119, 2027980013, 2031650022, 2031450026, 2032090045, 2037480059, 2029480025, 2029480028, 2029850023, 2030690024, 2030610070, 2028420029, 2030980050, 2030810013, 2048500051, 2030800082, 2048380020, 2048380021, 2028500040, 2028660074, 2029620044, 2028660072, 2028260024, 2024880036, 1018220052, 1018220053, 1021100063, 1021150034, 1021150053, 1021150054, 1018490038, 1015550004, 1020390061, 1020390001, 1020390003, 1020390064, 1021160061, 1021100068, 1021180011, 1021090017, 1018480053, 1008880045, 1018790051, 1016290013, 1020750043, 1020750049, 1020750053, 1016340022, 1019110031, 1019540046, 1020450059, 1020620009, 1019400040, 1019400041, 1020810010, 1020760010, 1020450069, 1020630004, 1020820045, 1020830063, 1021270043, 1020310035, 1004100012, 1016310140, 1020600012, 1019880042, 1019880046, 1020600014, 1020600052, 1018430026, 1021360040, 1021360043, 1021360052, 1020580050, 1021310052, 1019930092, 1016880037, 1017280001, 1020540016, 1020640043, 1017510044, 1019060057, 1019080056, 1017270014, 1017270016, 1017270020, 1017270010, 2032250091, 2027630195, 1017180027, 1017210003, 1003770026, 1021660069, 1020640149, 4017040167, 4017190030, 4002970038, 4088790027, 4101620071, 4122060026, 4017450107, 4157580007, 4157610003, 4158170048, 4158170148, 4157050009, 4157050013, 3005390025, 4113380015, 4159540052, 4121080013, 4159620005, 4159620049, 5000440046, 5010120082, 5012360081, 5000480011, 5001730042, 5000590029, 5001030028, 5001780080, 5001320003]
len(transfers)

530

In [3]:
r10 = pd.read_excel(r"C:\Users\xier\OneDrive - NYC O365 HOSTED\Analytics\02 - DEV\Strategic Preservation\Third Party Transfer\01 - Ongoing Projects\TPT Criteria Update\1. Inputs\DRAFT v6 - TPT Transferred Props Rounds 8-10 - FOR DISCUSSION ONLY - PRIVILEGED AND CONFIDENTIAL.xlsx", sheet_name = 2)

In [4]:
# get BBLs for Round X transfers
r10 = r10[r10.Round == 10]
len(r10)

62

In [5]:
r10 = r10['BBL'].astype(np.int64).tolist()

In [6]:
# add 1463 Amsterdam to the list of transfers
r10.append(1019700068)

In [7]:
transfers.extend(r10)
len(transfers)

593

In [8]:
r1_9 = pd.read_excel(r'C:\Users\xier\OneDrive - NYC O365 HOSTED\Analytics\02 - DEV\Strategic Preservation\Third Party Transfer\01 - Ongoing Projects\TPT Criteria Update\1. Inputs\Rounds 1-9 Public View 10 18 13 - Copy.xlsx')

In [9]:
r1_9.BBL.info()

<class 'pandas.core.series.Series'>
RangeIndex: 530 entries, 0 to 529
Series name: BBL
Non-Null Count  Dtype
--------------  -----
530 non-null    int64
dtypes: int64(1)
memory usage: 4.3 KB


In [10]:
# convert BBL list to df
df = pd.DataFrame(transfers)

In [11]:
df.columns = ['BBL']

In [12]:
len(df)

593

In [13]:
df.duplicated().sum()

0

In [14]:
df = df.merge(r1_9[['BBL', 'RD']], on ='BBL', how = 'left')

In [15]:
df[df.duplicated()]

,BBL,RD


In [16]:
df.RD.value_counts()

2     93
4     84
9     76
3     75
1     48
5B    48
6     37
5     36
7     32
5C     1
Name: RD, dtype: int64

In [17]:
len(df)

593

In [18]:
df.RD = np.where(df.RD.isna(), "10", df.RD)

In [19]:
df.RD.value_counts()

2     93
4     84
9     76
3     75
10    63
1     48
5B    48
6     37
5     36
7     32
5C     1
Name: RD, dtype: int64